In [1]:
import os
import rospy
import numpy as np
from sklearn.covariance import empirical_covariance, shrunk_covariance, oas, ledoit_wolf
from sklearn import linear_model
import cPickle as pickle
import seaborn as sns
import matplotlib.pyplot as plt
from rospy_service_helper import get_task_trajectories
from ll4ma_movement_primitives.promps import ProMP, ProMPConfig

lib_file = os.path.expanduser("~/.rosbags/demos/promp_backup/promp_lib_062.pkl")
with open(lib_file, 'r') as f:
    lib = pickle.load(f)
promp = lib.get_promp("DEFAULT")
promp.w.shape
rospy.init_node("test_cov")

In [ ]:
emp_cov = empirical_covariance(promp.w[:,:30], assume_centered=False)
plt.figure()
sns.heatmap(emp_cov)
plt.show()
# e_vals, _ = np.linalg.eig(emp_cov)
# print "EIG", e_vals[e_vals > 1e-10]
print "DET", np.linalg.det(emp_cov)
print "INV", np.linalg.inv(emp_cov)

In [2]:
def damped_pinv(A, rho=0.1):
    AA_T = np.dot(A, A.T)
    damping = np.eye(A.shape[0]) * rho**2
    inv = np.linalg.inv(AA_T + damping)
    d_pinv = np.dot(A.T, inv)
    return d_pinv

def gaussian(x, mu, sigma_inv):
    k = len(mu)
    exp_arg = -0.5 * np.dot((x - mu).T, np.dot(sigma_inv, x - mu))
    coef = 1.0 / np.sqrt((2*np.pi)**k) * np.sqrt(np.linalg.det(sigma_inv))
    return coef * np.exp(exp_arg)

In [ ]:
num_samples = 100
ps = None
for i in range(num_samples):
    traj = promp.generate_trajectory()[0]
    p = np.array([traj['x'][0][-1], traj['x'][1][-1], traj['x'][2][-1]])
    ps = p if ps is None else np.vstack((ps, p))


emp_mu = np.mean(ps, axis=0)
emp_cov = empirical_covariance(ps, assume_centered=False)
# emp_cov = shrunk_covariance(emp_cov, shrinkage=0.1)

# plt.figure()
# sns.heatmap(emp_cov)
# plt.show()

# sign, logdet = np.linalg.slogdet(emp_cov)

print "DET", np.linalg.det(emp_cov)
# print "OTHER DET", np.exp(logdet)
print "EMP", emp_cov
print "INV", np.linalg.inv(emp_cov)
print "PINV", np.linalg.pinv(emp_cov)

rho = 0.05
d_pinv = damped_pinv(emp_cov, rho)
print "DAMPED PINV", d_pinv
print "DAMPED DET", np.linalg.det(d_pinv)

e_vals, e_vecs = np.linalg.eig(d_pinv)

In [ ]:
emp_cov = promp.get_sigma_traj(1.0)
print emp_cov

rho = 0.017
sample = emp_mu - 0.0
sigma_inv = damped_pinv(emp_cov, rho)
gaussian(sample, emp_mu, sigma_inv)

In [3]:
def get_data_trajs(j_trajs=[], ee_trajs=[], obj_trajs=[]):
    num_trajectories = max(len(j_trajs), len(ee_trajs), len(obj_trajs))
    data = {}
    for i in range(num_trajectories):
        data_i = {}
        trajectory_name = None
        # Add joint trajectory data
        if j_trajs:
            qs = dqs = xs = obj_xs = None
            for j_point in j_trajs[i].points:
                q = np.array(j_point.positions)[:,None]
                qs =  q if qs is None else np.hstack((qs, q))
                dq = np.array(j_point.velocities)[:,None]
                dqs = dq if dqs is None else np.hstack((dqs, dq))
            data_i['q']    = qs
            data_i['qdot'] = dqs
            trajectory_name = j_trajs[i].trajectory_name
        # Add end-effector trajectory
        if ee_trajs:
            xs = None
            for ee_point in ee_trajs[i].points:
                x = []
                x.append(ee_point.pose.position.x)
                x.append(ee_point.pose.position.y)
                x.append(ee_point.pose.position.z)
                x.append(ee_point.pose.orientation.x)
                x.append(ee_point.pose.orientation.y)
                x.append(ee_point.pose.orientation.z)
                x.append(ee_point.pose.orientation.w)
                x = np.array(x).reshape((7,1))
                xs = x if xs is None else np.hstack((xs, x))
            data_i['x'] = xs[:3,:] # TODO TEMPORARY
            trajectory_name = ee_trajs[i].trajectory_name
        # Add object trajectory
        if obj_trajs:
            obj_xs = None
            for obj_point in obj_trajs[i].points:
                obj_x = []
                obj_x.append(obj_point.pose.position.x)
                obj_x.append(obj_point.pose.position.y)
                obj_x.append(obj_point.pose.position.z)
                obj_x.append(obj_point.pose.orientation.x)
                obj_x.append(obj_point.pose.orientation.y)
                obj_x.append(obj_point.pose.orientation.z)
                obj_x.append(obj_point.pose.orientation.w)
                obj_x = np.array(obj_x).reshape((7,1))
                obj_xs = obj_x if obj_xs is None else np.hstack((obj_xs, obj_x))
            data_i['obj_x'] = obj_xs[:3,:] # TODO TEMPORARY
            trajectory_name = obj_trajs[i].trajectory_name
        data[trajectory_name] = data_i
    return data

def get_config_from_data(data):
    config = ProMPConfig()
    demos = []
    for demo_name in data.keys():
        ds = None
        config.state_types = []
        config.dimensions = []
        for state_type in data[demo_name].keys():
            dimensions = []
            for dim in range(data[demo_name][state_type].shape[0]):
                dimensions.append(dim)
                d = data[demo_name][state_type][dim][:,None]
                ds = d if ds is None else np.vstack((ds,d))
            config.state_types.append(state_type)
            config.dimensions.append(dimensions)
        demos.append(ds)
    return config, demos


traj_idxs = range(58,68)
names = ["trajectory_%03d" % idx for idx in traj_idxs]
t_trajs = get_task_trajectories("/pandas/get_task_trajectories", "end_effector", names)
data = get_data_trajs(ee_trajs=t_trajs)

In [33]:
regr = linear_model.Ridge(1e-16, fit_intercept=True, normalize=True)
config, demos = get_config_from_data(data)
promp = ProMP(config=config)
ws = None
for demo in demos:
    w = promp.learn_weights(demo, regr=regr).flatten()
    ws = w if ws is None else np.vstack((ws, w))
    
# Sample mean
mu_w_mle = np.mean(ws, axis=0)[:,None]

# ws -= mu_w_mle.reshape((1,len(mu_w_mle)))

# Sample covariance
sigma_w_mle = np.zeros((len(mu_w_mle), len(mu_w_mle)))
num_samples = ws.shape[0]
for i in range(num_samples):
    sample_w = ws[i,:][:,None]
    sigma_w_mle += np.outer(sample_w - mu_w_mle, sample_w - mu_w_mle)
sigma_w_mle /= num_samples

phi, keys = promp.get_block_phi(1.0)

emp_cov = np.dot(phi, np.dot(sigma_w_mle, phi.T))
emp_mu  = np.dot(phi, mu_w_mle)

sigma_inv = np.linalg.pinv(emp_cov)

print ""
print "DET", np.linalg.det(emp_cov)
print "EMP", emp_cov
print "INV", sigma_inv
print "PINV", np.linalg.pinv(emp_cov)

[INFO] [1535589434.635064]: [ProMP] Initializing.
[INFO] [1535589434.635751]: [ProMP] 

State:
    x : [0, 1, 2]

[INFO] [1535589434.636285]: [ProMP] Initialization complete.

DET 0.0722828234427
EMP [[ 1486.7893369   1486.34033103  1486.34253063]
 [ 1486.34033103  1485.90714075  1485.90616013]
 [ 1486.34253063  1485.90616013  1485.90892482]]
INV [[  76.99156304  -11.63316592  -65.38088581]
 [ -11.63316592  268.75938302 -257.12232235]
 [ -65.38088581 -257.12232236  322.52248167]]
PINV [[  76.99156304  -11.63316592  -65.38088581]
 [ -11.63316592  268.75938302 -257.12232235]
 [ -65.38088581 -257.12232236  322.52248167]]


/usr/local/lib/python2.7/dist-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 3.04779325961e-17
  ' condition number: {}'.format(rcond), RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 7.51267768404e-17
  ' condition number: {}'.format(rcond), RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 3.47406087502e-18
  ' condition number: {}'.format(rcond), RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/scipy/linalg/basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to b

In [34]:
sample = emp_mu
gaussian(sample, emp_mu, sigma_inv)

array([[ 0.23616343]])